In [1]:
import pandas as pd
import numpy as np 
import scipy.stats as stats #统计模块
import scipy
#import pymysql #导入数据库模块

from datetime import datetime #时间模块
import statsmodels.formula.api as smf # OLS regression

import matplotlib.pyplot as plt #画图模块
import matplotlib.dates as mdates #日期格式处理模块
from matplotlib import style #自定义图表风格

from matplotlib.font_manager import FontProperties #作图中文
from pylab import mpl #画图自定义字体
#mpl.rcParams['font.sans-serif']=['SimHei']#指定默认字体
#mpl.rcParams['font.family']='Times New Roman'#指定默认字体

#输出矢量图 渲染矢量图
%matplotlib inline
%config InlineBackend.figure_format='svg' #提高分辨率

from IPython.core.interactiveshell import InteractiveShell #jupyter运行输出的模块
#显示每一个运行的结果
InteractiveShell.ast_node_interactivity='all'

#设置行不限制数量
#pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns',None)

In [2]:
cross = pd.read_csv("D:/python data/ret_mon_python2023.csv")
from pandas.tseries.offsets import MonthEnd

cross['month'] = pd.to_datetime(cross['month'],format='%Y-%m-%d')+MonthEnd(1)
# 补齐股票代码 如果不满6位 在前面补上0
cross['Stkcd']=cross['Stkcd'].apply(lambda x:'{:0>6}'.format(x))#6位股票代码
cross['w']=1
cross=cross.dropna(subset=['ret'])
cross

,Stkcd,month,price,Rank,Freq,floatingvalue,totalvalue,sizef,sizet,Return,rfmonth,ret,next_ret,w
1,000001,1991-05-31,38.34,2,24.0,1.016010e+09,1.859497e+09,20.739149,21.343572,-0.122253,0.006092,-0.128345,-0.119551,1
2,000001,1991-06-30,33.99,3,23.0,9.007350e+08,1.648521e+09,20.618722,21.223144,-0.113459,0.006092,-0.119551,-0.137013,1
3,000001,1991-07-31,29.54,4,16.0,7.828100e+08,1.432695e+09,20.478401,21.082823,-0.130921,0.006092,-0.137013,-0.417680,1
4,000001,1991-08-31,15.00,5,15.0,6.748338e+08,1.346275e+09,20.329977,21.020607,-0.411588,0.006092,-0.417680,-0.039425,1
5,000001,1991-09-30,14.50,6,24.0,6.523394e+08,1.301399e+09,20.296075,20.986706,-0.033333,0.006092,-0.039425,0.849080,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752023,605599,2023-08-31,13.56,24,23.0,7.955351e+09,1.054667e+10,22.797111,23.079076,-0.025862,0.001241,-0.027103,0.071030,1
752024,605599,2023-09-30,14.54,25,20.0,8.530295e+09,1.130889e+10,22.866890,23.148855,0.072271,0.001241,0.071030,-0.048696,1
752025,605599,2023-10-31,13.85,26,17.0,8.125488e+09,1.077222e+10,22.818272,23.100237,-0.047455,0.001241,-0.048696,-0.027956,1
752026,605599,2023-11-30,13.48,27,22.0,7.908417e+09,1.048444e+10,22.791193,23.073159,-0.026715,0.001241,-0.027956,0.103358,1


- 股市整体收益率：上证指数
- 股票市场整体收益率：所有股票来计算（包含0，3，6开头）
- 3开头是深圳创业板 2009开始

$$
MKT_t = R_{m,t} - R_{f,t}
$$

In [3]:
Market_ret = pd.read_excel("D:/python data/Marketret_mon_stock2023.xlsx")
Market_ret['month'] = pd.to_datetime(Market_ret['month'],format='%b %Y') + MonthEnd(1)
Market_ret.rename(columns={'ret':'MKT'}, inplace=True)
Market_ret

,month,MarketR,MarketR_e,rfmonth,MKT,ret_e,marketret3,marketret6,marketret12,Q
0,1991-01-31,0.029998,0.036554,0.006930,0.023068,0.029624,-0.084127,-0.305662,0.254049,1991 Q1
1,1991-02-28,0.010203,0.021860,0.006930,0.003273,0.014930,-0.183573,-0.384745,0.241492,1991 Q1
2,1991-03-31,-0.099663,-0.060425,0.006930,-0.106593,-0.067355,-0.252928,-0.445049,0.288857,1991 Q1
3,1991-04-30,-0.079652,-0.031449,0.006651,-0.086303,-0.038100,-0.234776,-0.394937,0.691749,1991 Q2
4,1991-05-31,-0.074521,0.005375,0.006092,-0.080613,-0.000717,-0.236294,0.181673,1.542701,1991 Q2
...,...,...,...,...,...,...,...,...,...,...
391,2023-08-31,-0.055570,-0.041538,0.001241,-0.056811,-0.042779,-0.089387,NaN,NaN,2023 Q3
392,2023-09-30,-0.005689,0.000357,0.001241,-0.006930,-0.000884,-0.033855,NaN,NaN,2023 Q3
393,2023-10-31,-0.026321,-0.009959,0.001241,-0.027562,-0.011200,-0.047817,NaN,NaN,2023 Q4
394,2023-11-30,0.001790,0.034280,0.001241,0.000549,0.033039,NaN,NaN,NaN,2023 Q4


In [4]:
cross=pd.merge(cross,Market_ret[['month','MKT']] ,on='month',how='left')
cross

,Stkcd,month,price,Rank,Freq,floatingvalue,totalvalue,sizef,sizet,Return,rfmonth,ret,next_ret,w,MKT
0,000001,1991-05-31,38.34,2,24.0,1.016010e+09,1.859497e+09,20.739149,21.343572,-0.122253,0.006092,-0.128345,-0.119551,1,-0.080613
1,000001,1991-06-30,33.99,3,23.0,9.007350e+08,1.648521e+09,20.618722,21.223144,-0.113459,0.006092,-0.119551,-0.137013,1,-0.085440
2,000001,1991-07-31,29.54,4,16.0,7.828100e+08,1.432695e+09,20.478401,21.082823,-0.130921,0.006092,-0.137013,-0.417680,1,-0.088189
3,000001,1991-08-31,15.00,5,15.0,6.748338e+08,1.346275e+09,20.329977,21.020607,-0.411588,0.006092,-0.417680,-0.039425,1,-0.090277
4,000001,1991-09-30,14.50,6,24.0,6.523394e+08,1.301399e+09,20.296075,20.986706,-0.033333,0.006092,-0.039425,0.849080,1,-0.031046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709878,605599,2023-08-31,13.56,24,23.0,7.955351e+09,1.054667e+10,22.797111,23.079076,-0.025862,0.001241,-0.027103,0.071030,1,-0.056811
709879,605599,2023-09-30,14.54,25,20.0,8.530295e+09,1.130889e+10,22.866890,23.148855,0.072271,0.001241,0.071030,-0.048696,1,-0.006930
709880,605599,2023-10-31,13.85,26,17.0,8.125488e+09,1.077222e+10,22.818272,23.100237,-0.047455,0.001241,-0.048696,-0.027956,1,-0.027562
709881,605599,2023-11-30,13.48,27,22.0,7.908417e+09,1.048444e+10,22.791193,23.073159,-0.026715,0.001241,-0.027956,0.103358,1,0.000549


In [5]:
import statsmodels.api as sm

# 假设我们有一个DataFrame 'data'，其中包含个股和市场的月度收益率
# 'data' 包含四列：'month'（月份），'Stkcd'（股票代码），'stock_return'（个股收益率），'market_return'（市场收益率）

# 定义一个函数来计算滚动回归的贝塔和使用的数据点数量
def rolling_beta_per_stock(data, window_months=60):
    betas = []
    months = []
    Stkcds = []
    data_counts = []
    
    # 按股票分组
    grouped = data.groupby('Stkcd')
    
    for Stkcd, group in grouped:
        group = group.set_index('month').sort_index()
        end_months = group.index.unique()
        
        for end_month in end_months:
            start_month = end_month - pd.DateOffset(months=window_months)
            window_data = group.loc[start_month:end_month]
            
            if len(window_data) > 0:
                X = sm.add_constant(window_data['MKT'])
                y = window_data['ret']
                model = sm.OLS(y, X).fit()
                
                beta = model.params['MKT']
                betas.append(beta)
                months.append(end_month)
                Stkcds.append(Stkcd)
                data_counts.append(len(window_data))  # 记录使用的数据点数量
    
    return pd.DataFrame({'Stkcd': Stkcds, 'month': months, 'beta': betas, 'data_count': data_counts})

In [6]:
rolling_beta_per_stock(cross,window_months=60)
rolling_beta_per_stock

,Stkcd,month,beta,data_count
0,000001,1991-05-31,1.592113,1
1,000001,1991-06-30,-1.821836,2
2,000001,1991-07-31,0.807037,3
3,000001,1991-08-31,23.378197,4
4,000001,1991-09-30,3.358613,5
...,...,...,...,...
709878,605599,2023-08-31,1.040511,23
709879,605599,2023-09-30,1.038189,24
709880,605599,2023-10-31,1.052476,25
709881,605599,2023-11-30,1.047631,26


<function __main__.rolling_beta_per_stock(data, window_months=60)>

In [8]:
rolling_beta=pd.read_csv("D:/python data/rolling_betas.csv")
rolling_beta['month']=pd.to_datetime(rolling_beta['month'],format='%Y-%m-%d')
rolling_beta['Stkcd']=rolling_beta['Stkcd'].apply(lambda x:'{:0>6}'.format(x))
rolling_beta

,Stkcd,month,beta,data_count
0,000001,1991-05-31,1.592113,1
1,000001,1991-06-30,-1.821836,2
2,000001,1991-07-31,0.807037,3
3,000001,1991-08-31,23.378197,4
4,000001,1991-09-30,3.358613,5
...,...,...,...,...
709878,605599,2023-08-31,1.040511,23
709879,605599,2023-09-30,1.038189,24
709880,605599,2023-10-31,1.052476,25
709881,605599,2023-11-30,1.047631,26


In [9]:
cross_beta=pd.merge(cross,rolling_beta,on=['Stkcd','month'],how='left')
cross_beta=cross_beta[cross_beta['data_count']>=12]
cross_beta

,Stkcd,month,price,Rank,Freq,floatingvalue,totalvalue,sizef,sizet,Return,rfmonth,ret,next_ret,w,MKT,beta,data_count
11,000001,1992-04-30,38.50,13,22.0,2.599445e+09,5.185820e+09,21.678564,22.369194,0.477927,0.006092,0.471835,0.167934,1,0.356928,1.287898,12
12,000001,1992-05-31,45.20,14,21.0,3.051816e+09,6.088287e+09,21.839003,22.529633,0.174026,0.006092,0.167934,-0.076888,1,0.698773,0.903993,13
13,000001,1992-06-30,42.00,15,22.0,2.835758e+09,5.657258e+09,21.765575,22.456205,-0.070796,0.006092,-0.076888,0.079622,1,-0.112863,0.886484,14
14,000001,1992-07-31,45.60,16,23.0,3.078823e+09,6.142166e+09,21.847813,22.538443,0.085714,0.006092,0.079622,-0.101487,1,0.030265,0.878303,15
15,000001,1992-08-31,41.25,17,21.0,2.785120e+09,5.556236e+09,21.747557,22.438187,-0.095395,0.006092,-0.101487,0.075120,1,-0.152259,0.862723,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709878,605599,2023-08-31,13.56,24,23.0,7.955351e+09,1.054667e+10,22.797111,23.079076,-0.025862,0.001241,-0.027103,0.071030,1,-0.056811,1.040511,23
709879,605599,2023-09-30,14.54,25,20.0,8.530295e+09,1.130889e+10,22.866890,23.148855,0.072271,0.001241,0.071030,-0.048696,1,-0.006930,1.038189,24
709880,605599,2023-10-31,13.85,26,17.0,8.125488e+09,1.077222e+10,22.818272,23.100237,-0.047455,0.001241,-0.048696,-0.027956,1,-0.027562,1.052476,25
709881,605599,2023-11-30,13.48,27,22.0,7.908417e+09,1.048444e+10,22.791193,23.073159,-0.026715,0.001241,-0.027956,0.103358,1,0.000549,1.047631,26


In [10]:
fenweishu = pd.DataFrame(
    cross_beta.groupby(['month'])['beta'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))
fenweishu = fenweishu.reset_index()
fenweishu = fenweishu.pivot_table(index='month',columns='level_1',values='beta')
fenweishu.columns = ['one','two','three','four','five','six','seven','eight','nine']
fenweishu

,one,two,three,four,five,six,seven,eight,nine
month,,,,,,,,,
1991-12-31,0.071958,0.097514,0.099366,0.101218,0.149016,0.196814,0.221577,0.246341,0.253227
1992-01-31,0.082628,0.094319,0.097810,0.157148,0.221045,0.251824,0.263519,0.573773,1.928159
1992-02-29,0.081935,0.093897,0.101806,0.168023,0.228162,0.251692,0.263399,0.573994,1.927546
1992-03-31,0.061369,0.078081,0.093707,0.157350,0.216032,0.240144,0.260510,0.586451,1.895887
1992-04-30,-0.065538,-0.007246,0.048498,0.063799,0.158559,0.263182,0.443521,1.287898,1.787884
...,...,...,...,...,...,...,...,...,...
2023-08-31,0.577038,0.746623,0.860449,0.956527,1.038594,1.132528,1.239638,1.380807,1.601730
2023-09-30,0.559448,0.731075,0.849778,0.942773,1.023284,1.124444,1.233064,1.381506,1.600269
2023-10-31,0.557654,0.731994,0.849838,0.943937,1.024999,1.121929,1.232482,1.381136,1.603640


In [11]:
portfolio = pd.merge(cross_beta,fenweishu,on='month')
portfolio

,Stkcd,month,price,Rank,Freq,floatingvalue,totalvalue,sizef,sizet,Return,rfmonth,ret,next_ret,w,MKT,beta,data_count,one,two,three,four,five,six,seven,eight,nine
0,000001,1992-04-30,38.50,13,22.0,2.599445e+09,5.185820e+09,21.678564,22.369194,0.477927,0.006092,0.471835,0.167934,1,0.356928,1.287898,12,-0.065538,-0.007246,0.048498,0.063799,0.158559,0.263182,0.443521,1.287898,1.787884
1,000001,1992-05-31,45.20,14,21.0,3.051816e+09,6.088287e+09,21.839003,22.529633,0.174026,0.006092,0.167934,-0.076888,1,0.698773,0.903993,13,0.050273,0.126902,0.160976,0.190613,0.293657,0.505642,0.838514,0.903993,1.373230
2,000001,1992-06-30,42.00,15,22.0,2.835758e+09,5.657258e+09,21.765575,22.456205,-0.070796,0.006092,-0.076888,0.079622,1,-0.112863,0.886484,14,0.109463,0.186703,0.212601,0.220830,0.374333,0.738099,0.888928,1.064494,1.327165
3,000001,1992-07-31,45.60,16,23.0,3.078823e+09,6.142166e+09,21.847813,22.538443,0.085714,0.006092,0.079622,-0.101487,1,0.030265,0.878303,15,0.116811,0.190736,0.217469,0.237481,0.525851,0.779806,0.878420,1.015655,1.295484
4,000001,1992-08-31,41.25,17,21.0,2.785120e+09,5.556236e+09,21.747557,22.438187,-0.095395,0.006092,-0.101487,0.075120,1,-0.152259,0.862723,16,0.179535,0.277557,0.282518,0.398499,0.670156,0.828769,0.909351,1.059466,1.292737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658177,605599,2023-08-31,13.56,24,23.0,7.955351e+09,1.054667e+10,22.797111,23.079076,-0.025862,0.001241,-0.027103,0.071030,1,-0.056811,1.040511,23,0.577038,0.746623,0.860449,0.956527,1.038594,1.132528,1.239638,1.380807,1.601730
658178,605599,2023-09-30,14.54,25,20.0,8.530295e+09,1.130889e+10,22.866890,23.148855,0.072271,0.001241,0.071030,-0.048696,1,-0.006930,1.038189,24,0.559448,0.731075,0.849778,0.942773,1.023284,1.124444,1.233064,1.381506,1.600269
658179,605599,2023-10-31,13.85,26,17.0,8.125488e+09,1.077222e+10,22.818272,23.100237,-0.047455,0.001241,-0.048696,-0.027956,1,-0.027562,1.052476,25,0.557654,0.731994,0.849838,0.943937,1.024999,1.121929,1.232482,1.381136,1.603640
658180,605599,2023-11-30,13.48,27,22.0,7.908417e+09,1.048444e+10,22.791193,23.073159,-0.026715,0.001241,-0.027956,0.103358,1,0.000549,1.047631,26,0.546811,0.720276,0.839969,0.934261,1.019290,1.115750,1.230708,1.384028,1.605226


In [12]:
portfolio['sort'] = np.where(
    portfolio['beta'] <= portfolio['one'], 'P1',
    np.where(
        portfolio['beta'] <= portfolio['two'], 'P2',
        np.where(
            portfolio['beta'] <= portfolio['three'], 'P3',
            np.where(
                portfolio['beta'] <= portfolio['four'], 'P4',
                np.where(
                    portfolio['beta'] <= portfolio['five'], 'P5',
                    np.where(
                        portfolio['beta'] <= portfolio['six'], 'P6',
                        np.where(
                            portfolio['beta'] <= portfolio['seven'], 'P7',
                            np.where(
                                portfolio['beta'] <= portfolio['eight'], 'P8',
                                np.where(
                                    portfolio['beta'] <= portfolio['nine'],
                                    'P9', 'Pmax')))))))))
portfolio = portfolio.dropna(subset=['floatingvalue','next_ret','beta'])
portfolio

,Stkcd,month,price,Rank,Freq,floatingvalue,totalvalue,sizef,sizet,Return,rfmonth,ret,next_ret,w,MKT,beta,data_count,one,two,three,four,five,six,seven,eight,nine,sort
0,000001,1992-04-30,38.50,13,22.0,2.599445e+09,5.185820e+09,21.678564,22.369194,0.477927,0.006092,0.471835,0.167934,1,0.356928,1.287898,12,-0.065538,-0.007246,0.048498,0.063799,0.158559,0.263182,0.443521,1.287898,1.787884,P8
1,000001,1992-05-31,45.20,14,21.0,3.051816e+09,6.088287e+09,21.839003,22.529633,0.174026,0.006092,0.167934,-0.076888,1,0.698773,0.903993,13,0.050273,0.126902,0.160976,0.190613,0.293657,0.505642,0.838514,0.903993,1.373230,P8
2,000001,1992-06-30,42.00,15,22.0,2.835758e+09,5.657258e+09,21.765575,22.456205,-0.070796,0.006092,-0.076888,0.079622,1,-0.112863,0.886484,14,0.109463,0.186703,0.212601,0.220830,0.374333,0.738099,0.888928,1.064494,1.327165,P7
3,000001,1992-07-31,45.60,16,23.0,3.078823e+09,6.142166e+09,21.847813,22.538443,0.085714,0.006092,0.079622,-0.101487,1,0.030265,0.878303,15,0.116811,0.190736,0.217469,0.237481,0.525851,0.779806,0.878420,1.015655,1.295484,P7
4,000001,1992-08-31,41.25,17,21.0,2.785120e+09,5.556236e+09,21.747557,22.438187,-0.095395,0.006092,-0.101487,0.075120,1,-0.152259,0.862723,16,0.179535,0.277557,0.282518,0.398499,0.670156,0.828769,0.909351,1.059466,1.292737,P7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658176,605599,2023-07-31,13.92,23,21.0,8.166555e+09,1.082667e+10,22.823313,23.105278,0.053747,0.001241,0.052506,-0.027103,1,0.035528,1.059668,22,0.587592,0.755658,0.866035,0.962244,1.050305,1.140063,1.252768,1.391693,1.614129,P6
658177,605599,2023-08-31,13.56,24,23.0,7.955351e+09,1.054667e+10,22.797111,23.079076,-0.025862,0.001241,-0.027103,0.071030,1,-0.056811,1.040511,23,0.577038,0.746623,0.860449,0.956527,1.038594,1.132528,1.239638,1.380807,1.601730,P6
658178,605599,2023-09-30,14.54,25,20.0,8.530295e+09,1.130889e+10,22.866890,23.148855,0.072271,0.001241,0.071030,-0.048696,1,-0.006930,1.038189,24,0.559448,0.731075,0.849778,0.942773,1.023284,1.124444,1.233064,1.381506,1.600269,P6
658179,605599,2023-10-31,13.85,26,17.0,8.125488e+09,1.077222e+10,22.818272,23.100237,-0.047455,0.001241,-0.048696,-0.027956,1,-0.027562,1.052476,25,0.557654,0.731994,0.849838,0.943937,1.024999,1.121929,1.232482,1.381136,1.603640,P6


In [13]:
portfolio_beta =  pd.DataFrame(
    portfolio.groupby(['month','sort']).apply(lambda x: np.average(x['next_ret'],weights = x['floatingvalue']),include_groups=False))
portfolio_beta

0
month      sort          
1991-12-31 P1    0.143309
           P2    0.144150
           P4    0.155164
           P6    0.066497
           P8    0.087397
...                   ...
2023-11-30 P6   -0.024932
           P7   -0.027459
           P8   -0.024982
           P9   -0.026196
           Pmax -0.027796

[3835 rows x 1 columns]

In [14]:
portfolio_beta = portfolio_beta.reset_index()
portfolio_beta.columns = ['month', 'sort', 'p']
portfolio_beta['month'] = portfolio_beta['month'] + MonthEnd(1)
portfolio_beta

,month,sort,p
0,1992-01-31,P1,0.143309
1,1992-01-31,P2,0.144150
2,1992-01-31,P4,0.155164
3,1992-01-31,P6,0.066497
4,1992-01-31,P8,0.087397
...,...,...,...
3830,2023-12-31,P6,-0.024932
3831,2023-12-31,P7,-0.027459
3832,2023-12-31,P8,-0.024982
3833,2023-12-31,P9,-0.026196


In [16]:
portfolio_beta = portfolio_beta.pivot_table(index='month',
                                            columns='sort',
                                            values='p')
portfolio_beta['My_portfolio'] = portfolio_beta['P1'] - portfolio_beta['Pmax']
portfolio_beta

sort,P1,P2,P3,P4,P5,P6,P7,P8,P9,Pmax,My_portfolio
month,,,,,,,,,,,
1992-01-31,0.143309,0.144150,NaN,0.155164,NaN,0.066497,NaN,0.087397,NaN,0.024295,0.119014
1992-02-29,0.104298,0.016538,0.067644,0.853761,1.923706,0.081949,0.003954,0.017056,-0.047933,-0.055072,0.159370
1992-03-31,0.136385,0.014243,0.044922,0.122783,0.139400,0.055180,0.003962,0.108161,0.103078,0.090903,0.045482
1992-04-30,-0.155729,0.137519,0.049913,-0.089302,-0.390592,-0.036502,0.599388,-0.016244,0.647451,0.594847,-0.750576
1992-05-31,0.337114,0.365337,0.582005,1.582324,0.175726,-0.000964,0.473241,0.167934,0.445337,0.489509,-0.152395
...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,-0.036412,-0.054473,-0.068013,-0.057719,-0.068990,-0.058171,-0.054705,-0.059133,-0.063060,-0.071705,0.035293
2023-09-30,0.018065,0.006200,-0.007416,-0.010419,-0.004966,-0.001680,-0.017780,-0.035491,-0.016884,-0.018909,0.036975
2023-10-31,-0.026936,-0.035263,-0.035169,-0.045496,-0.020575,-0.030723,-0.040041,-0.039563,-0.019571,0.002289,-0.029226


In [17]:
portfolio_beta = portfolio_beta['1995-01':'2023-12']
portfolio_beta

sort,P1,P2,P3,P4,P5,P6,P7,P8,P9,Pmax,My_portfolio
month,,,,,,,,,,,
1995-01-31,-0.119180,-0.072110,-0.130556,-0.124243,-0.120475,-0.126131,-0.135343,-0.145437,-0.152750,-0.155980,0.036800
1995-02-28,-0.009711,-0.008694,-0.010877,-0.012167,-0.006294,-0.027947,-0.032387,-0.039242,-0.041955,-0.051491,0.041780
1995-03-31,0.062154,0.058339,0.075692,0.104134,0.089152,0.138383,0.118963,0.147348,0.184887,0.218689,-0.156534
1995-04-30,-0.129923,-0.109784,-0.122973,-0.096805,-0.116945,-0.130956,-0.096548,-0.094319,-0.111986,-0.121447,-0.008475
1995-05-31,0.078637,0.066627,0.099883,0.103732,0.156507,0.220783,0.199680,0.197807,0.245065,0.223204,-0.144567
...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,-0.036412,-0.054473,-0.068013,-0.057719,-0.068990,-0.058171,-0.054705,-0.059133,-0.063060,-0.071705,0.035293
2023-09-30,0.018065,0.006200,-0.007416,-0.010419,-0.004966,-0.001680,-0.017780,-0.035491,-0.016884,-0.018909,0.036975
2023-10-31,-0.026936,-0.035263,-0.035169,-0.045496,-0.020575,-0.030723,-0.040041,-0.039563,-0.019571,0.002289,-0.029226


In [18]:
model = smf.ols('My_portfolio ~ 1',
                 data=portfolio_beta['1995-01':'2023-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           My_portfolio   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Dec 2024   Prob (F-statistic):                nan
Time:                        19:09:56   Log-Likelihood:                 474.65
No. Observations:                 348   AIC:                            -947.3
Df Residuals:                     347   BIC:                            -943.4
Df Model:                           0                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0071      0.003      2.247      0.0

- CAPM fails
  - Low-Risk Anomally: Returns are not positively correlated to stock betas/risk
  - $ \alpha < 0$

In [19]:
MYPORT=pd.merge(portfolio_beta,Market_ret[['month','MKT']],on='month',how='left')
MYPORT

,month,P1,P2,P3,P4,P5,P6,P7,P8,P9,Pmax,My_portfolio,MKT
0,1995-01-31,-0.119180,-0.072110,-0.130556,-0.124243,-0.120475,-0.126131,-0.135343,-0.145437,-0.152750,-0.155980,0.036800,-0.125017
1,1995-02-28,-0.009711,-0.008694,-0.010877,-0.012167,-0.006294,-0.027947,-0.032387,-0.039242,-0.041955,-0.051491,0.041780,-0.022778
2,1995-03-31,0.062154,0.058339,0.075692,0.104134,0.089152,0.138383,0.118963,0.147348,0.184887,0.218689,-0.156534,0.122163
3,1995-04-30,-0.129923,-0.109784,-0.122973,-0.096805,-0.116945,-0.130956,-0.096548,-0.094319,-0.111986,-0.121447,-0.008475,-0.113023
4,1995-05-31,0.078637,0.066627,0.099883,0.103732,0.156507,0.220783,0.199680,0.197807,0.245065,0.223204,-0.144567,0.166736
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2023-08-31,-0.036412,-0.054473,-0.068013,-0.057719,-0.068990,-0.058171,-0.054705,-0.059133,-0.063060,-0.071705,0.035293,-0.056811
344,2023-09-30,0.018065,0.006200,-0.007416,-0.010419,-0.004966,-0.001680,-0.017780,-0.035491,-0.016884,-0.018909,0.036975,-0.006930
345,2023-10-31,-0.026936,-0.035263,-0.035169,-0.045496,-0.020575,-0.030723,-0.040041,-0.039563,-0.019571,0.002289,-0.029226,-0.027562
346,2023-11-30,0.011564,0.005363,0.015737,0.014640,-0.001344,-0.022379,0.027644,-0.017196,-0.017168,-0.010074,0.021638,0.000549


In [20]:
# 回归策略到市场上
model = smf.ols('My_portfolio ~ MKT',
                 data=MYPORT).fit(cov_type='HAC', cov_kwds={'maxlags': 6})
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           My_portfolio   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     10.59
Date:                Tue, 10 Dec 2024   Prob (F-statistic):            0.00125
Time:                        19:10:49   Log-Likelihood:                 503.21
No. Observations:                 348   AIC:                            -1002.
Df Residuals:                     346   BIC:                            -994.7
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0095      0.003      3.343      0.0

- $$ R_{β,big} - R_{β,small} = -0.0095 + 0.2928(\beta_{big} - \beta_{small})

- CAPM fails
  - Cross sectional returns should be explained by their batas 100\%
  - Return cannot be predicted by any other variables
  - Size 规模 买小公司 卖大公司
  - Value 价值 买价值股 卖成长股